In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [0]:
dbutils.widgets.text('incremntal_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremntal_flag')

In [0]:
%sql

select * from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales`

In [0]:
df_source = spark.sql('select distinct(Dealer_ID),DealerName from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales`')
df_source.display()

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
    df_sink = spark.sql('select dim_dealer_key,Dealer_ID,DealerName from cars_catalog.gold.dim_dealer')
else:
    df_sink = spark.sql('select 0 dim_dealer_key,Dealer_ID,DealerName from delta.`abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales` where 1 = 0')

In [0]:
df_sink.display()

In [0]:
df_filter = df_source.join(df_sink,df_source.Dealer_ID == df_sink.Dealer_ID,'left').select(df_source.Dealer_ID,df_source.DealerName,df_sink.dim_dealer_key)
df_filter.display()


In [0]:
df_filter_new = df_filter.filter(df_filter.dim_dealer_key.isNull()).select(df_filter.Dealer_ID,df_filter.DealerName)
df_filter_old = df_filter.filter(df_filter.dim_dealer_key.isNotNull())

In [0]:
if (incremental_flag == '0'):
    max_value = 1
else:
    if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
        df_max_value = spark.sql('select max(dim_dealer_key) from cars_catalog.gold.dim_dealer')
        max_value = df_max_value.collect()[0][0]
if spark.catalog.tableExists('cars_catalog.silver.dim_dealer'):
    df_max_value = spark.sql('select max(dim_dealer_key) from cars_catalog.silver.dim_dealer')
    max_value = df_max_value.collect()[0][0]
else:
    max_value = 1


In [0]:
df_filter_new = df_filter_new.withColumn('dim_dealer_key',max_value+monotonically_increasing_id()+1)


In [0]:
df_final = df_filter_new.union(df_filter_old)

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
    delta_table = DeltaTable.forPath(spark,'abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_dealer')
    delta_table.alias('t').merge(df_final.alias('s'),'t.Dealer_ID = s.Dealer_ID').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_final.write.format('delta').mode('append').option('path','abfss://gold@azadlsgen7428.dfs.core.windows.net/dim_dealer').saveAsTable('cars_catalog.gold.dim_dealer')
